In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller, kpss

from scipy.stats import f_oneway

In [ ]:
df = pd.read_csv("../data/processed/mixed/features.csv")

In [ ]:
date = df["Unnamed: 0"]
df = df.drop("Unnamed: 0", axis=1)

ANOVA

In [ ]:
df["time2death"] = np.arange(df.shape[0], 0, -1)

In [ ]:
target = df["time2death"]
df = df.drop("time2death", axis=1)

In [ ]:
feat2stat = {}
for feat in df.columns:
    stat, pval = f_oneway(df[feat], target)
    feat2stat[feat] = stat
    print(f"feature: {feat}, f-statistic: {stat}, p-value: {pval}")

sns.barplot(feat2stat)
plt.xticks(rotation=90)

In [ ]:
stat, pval = f_oneway(*[df[col] for col in df.columns])
print(stat, pval)

Select features

In [ ]:
feat_names = ["kurtosis", "skewness", "mean", "shapeFactor", "2.Вибропреобразователь ППДв 5_1 СКЗ Виброускорение", "2.Вибропреобразователь ППДв 5_1 Рост СКЗ Виброускорения"]
df_selected = df[feat_names]

PCA

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected)

In [ ]:
pca = PCA(n_components=2)
pca2= pca.fit_transform(scaled_features)
sns.scatterplot(x=pca2[:, 0], y=pca2[:, 1])

In [ ]:
def calculate_monotonicity(df):
    n, m = df.shape
    monotonicities = []

    for column in df.columns:
        diffs = np.diff(df[column])
        num_positive_diffs = np.sum(diffs > 0)
        num_negative_diffs = np.sum(diffs < 0)
        monotonicity = np.abs(num_positive_diffs - num_negative_diffs) / (n - 1)
        monotonicities.append(monotonicity)

    return monotonicities

In [ ]:
monotonicities = calculate_monotonicity(df)
monotonicity_df = pd.DataFrame({'Signal': df.columns, 'Monotonicity': monotonicities})

monotonicity_df.sort_values("Monotonicity", ascending=False)

In [ ]:
sns.barplot(y=monotonicity_df["Signal"], x=monotonicity_df["Monotonicity"], orient='h')
plt.axvspan(xmin=0.015, xmax=.03, color="red", alpha=.5)

In [ ]:
feat_names = ["std", "energy", "2.Вибропреобразователь ППДв 5_1 СКЗ Виброускорение", "2.Вибропреобразователь ППДв 5_1 ПИК фактор"]
df_selected = df[feat_names]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected)

pca = PCA(n_components=2)
pca2= pca.fit_transform(scaled_features)
sns.scatterplot(x=pca2[:, 0], y=pca2[:, 1])

In [ ]:
df["RUL"] = target

In [ ]:
df.columns

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

corr_mat = df.corr()
mask_feature = np.triu(np.ones_like(corr_mat, dtype=bool))
heatmap = sns.heatmap(corr_mat, 
                      mask=mask_feature | (np.abs(corr_mat) < .5), 
                      annot=True, fmt=".2f", ax=ax)

xticklabels = heatmap.get_xticklabels()
yticklabels = heatmap.get_yticklabels()

for label in yticklabels:
    if len(label.get_text()) >= 15:
        label.set_visible(False)

heatmap.set_xticklabels(xticklabels)
heatmap.set_yticklabels(yticklabels)

plt.show()

In [ ]:
df.columns

In [ ]:
feat_names = ["peak2peak", "std", "energy", 
              "2.Вибропреобразователь ППДв 5_1 СКЗ Виброускорение", 
              "2.Вибропреобразователь ППДв 5_1 СКЗ Виброускорения 10 кГц",
              "2.Вибропреобразователь ППДв 5_1 Амплитудное значение виброускорения",
              "2.Вибропреобразователь ППДв 5_1 Размах виброперемещения",
              "2.Вибропреобразователь ППДв 5_1 ПИК фактор"]
df_selected = df[feat_names]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected)

pca = PCA(n_components=2)
pca2= pca.fit_transform(scaled_features)
sns.scatterplot(x=pca2[:, 0], y=pca2[:, 1])

In [ ]:
signals = df.drop("RUL", axis=1)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.figure(figsize=(15, 20))
for i, signal in enumerate(signals, 1):
    plt.subplot(13, 2, 2*i-1)
    plot_acf(signals[signal], lags=24, ax=plt.gca())
    plt.title(f'ACF для сигнала {signal[:50]}')

# Построение PACF для каждого сигнала
for i, signal in enumerate(signals, 1):
    plt.subplot(13, 2, 2*i)
    plot_pacf(signals[signal], lags=24, ax=plt.gca())
    plt.title(f'PACF для сигнала {signal[:50]}')

plt.tight_layout()
plt.show()

PACF - 1 лаг  
ACF -  0 

Много плавноубывающих признаков. Данные очень шумные. Автокорреляция бесполезная.

In [ ]:
feat_names = ["kurtosis", "skewness", "mean", "shapeFactor", "2.Вибропреобразователь ППДв 5_1 Рост СКЗ Виброускорения"]
df_selected = df[feat_names]

In [ ]:
for col in feat_names:
    result = adfuller(df_selected[col])
    print(f'ADF статистика для {col}: {result[0]}, p-value: {result[1]}')

for col in feat_names:
    result = kpss(df_selected[col])
    print(f'KPSS статистика для {col}: {result[0]}, p-value: {result[1]}')

Будем считать, что признаки стационарны

In [ ]:
model = VARMAX(df_selected, order=())  # Пример: модель с одним лагом и одним разностью
result = model.fit(1, 0)


НЕСТАЦИОНАРНЫЕ ОБРАТНО КОРЕЛЛИРОВАННЫЕ ПРИЗНАКИ

In [ ]:
feat_names = ["peak2peak", "std", "energy", 
              "2.Вибропреобразователь ППДв 5_1 СКЗ Виброускорение", 
              "2.Вибропреобразователь ППДв 5_1 СКЗ Виброускорения 10 кГц",
              "2.Вибропреобразователь ППДв 5_1 Амплитудное значение виброускорения",
              "2.Вибропреобразователь ППДв 5_1 Размах виброперемещения",
              "2.Вибропреобразователь ППДв 5_1 ПИК фактор"]
df_selected = df[feat_names]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected)

Добавление скользящих признаков

In [ ]:
# ADD EMA WITH DIFFERENT LENGTH
# LAGS


Регрессируем RUL (дефолтные параметры)

In [ ]:
# Split data
rul_model = GradientBoostingRegressor()

# Пробуем теперь по всем сигналам регрессировать
# Отобранные сигналы не дали нормального результата
feat_train, rul_train = scaled_features[:1000], df["RUL"].iloc[:1000]
feat_test, rul_test = scaled_features[1000:], df["RUL"].iloc[1000:]

rul_model.fit(feat_train, rul_train)

pred = rul_model.predict(feat_test)

print(f"rmse: {mean_squared_error(rul_test, pred, squared=False)}, mape: {mean_absolute_percentage_error(rul_test, pred)}, r2: {r2_score(rul_test, pred)}")


In [ ]:
sns.lineplot(pred)

In [ ]:
pred[-10:]